In [33]:
import pandas as pd
import numpy as np

from tqdm import tqdm
from config import *

In [144]:
train_df = pd.read_csv(data_path+"train.csv")
test_df = pd.read_csv(data_path+"test.csv")

In [119]:
train_dup = pd.read_json("input/dataset_with_original_text/train_duplicated_index.json", typ="series")
train_nofinds = pd.read_json("input/dataset_with_original_text/train_nofinds_index.json", typ="series")
test_dup = pd.read_json("input/dataset_with_original_text/test_duplicated_index.json", typ="series")
test_nofinds = pd.read_json("input/dataset_with_original_text/test_nofinds_index.json", typ="series")

In [36]:
# ##########################################
#
#               原文tsvの読み込み
#
# ##########################################
corpus_df_list = []
for source in ["newsplus", "news4vip", "livejupiter"]:
    corpus_df = pd.DataFrame(columns=["id", "source", "text"])
    tsv_line_list = []
    with open(input_root+f"corpus/{source}.tsv", encoding="utf-8") as f:
        for i, l in tqdm(enumerate(f)):
            tsv_line_list.append(l)
    corpus_df["text"] = tsv_line_list
    corpus_df["source"] = source
    corpus_df = corpus_df.reset_index(drop=False)
    corpus_df["id"] = corpus_df["source"] + "_" + corpus_df["index"].astype(str)
    corpus_df = corpus_df.drop(["index"], axis=1)
    corpus_df_list.append(corpus_df)

corpus_df = pd.concat(corpus_df_list)
corpus_df = corpus_df.reset_index(drop=True)

217296it [00:00, 1144538.90it/s]
1983626it [00:01, 1355912.03it/s]
5948218it [00:03, 1494261.19it/s]


In [138]:
def recover_original_text(df, corpus_df):

    original_text_list = []
    nofinds_data_index_list = []
    duplicated_data_index_list = []

    df["first_sentence"] = df["text"].map(lambda x: x.split("\n")[0])

    for i in tqdm(range(df.shape[0]), total=df.shape[0]):
        first_sentence = df.loc[i, "first_sentence"]
        text_picked = [x for x in corpus_df["text"].values.tolist() if first_sentence in x]

        if len(text_picked) == 0: # データセットはある程度正規化されているので、データ元からヒットしないケースがある --
            slicer = 1
            while (len(text_picked) == 0):  # 恐らくスペースが消えていることが原因、復元できないので、頭からスライスしていってヒットしたやつを採用する --
                all_sentence = df.loc[i, "text"]
                text_picked = [x for x in corpus_df["text"].values.tolist() if all_sentence[slicer:] in x]            
                slicer += 1
            nofinds_data_index_list.append(i)
            original_text_list.append(repr(text_picked[0]))

        elif len(text_picked) > 1: # 1つの文に対して2つ以上のレスアンカーがついていた場合, 2つ以上ヒットする --
            duplicated_data_index_list.append(i)  # 分離不能な場合があるので諦めてindexだけメモっときます --
            original_text_list.append(repr(text_picked[0]))

        else:  # 1つだけヒット
            original_text_list.append(repr(text_picked[0]))

    df["original_text"] = original_text_list
    df = df.drop(["first_sentence"], axis=1)

    return df, duplicated_data_index_list, nofinds_data_index_list

In [139]:
tdf = train_df.loc[[0,2,49]].reset_index(drop=True)

In [140]:
t, dup, nofinds = recover_original_text(tdf, corpus_df)

100%|██████████| 3/3 [00:16<00:00,  5.66s/it]


In [141]:
t

,id,source,text,label,original_text
0,80074aa43,news4vip,まともに相手されてない人との関係なんて\nそんな大事にするものか？,0,'まともにLINEの返信してくれないんだけど諦めなきゃダメだろ __BR__ フラれたらギク...
1,c535f5613,livejupiter,日本人として生まれても無能な低学歴って分かったら日本人の権利剥奪して追放すべきやろ\n甘えるな,1,'日本人として生まれても無能な低学歴って分かったら日本人の権利剥奪して追放すべきやろ __B...
2,c32c6dbb4,news4vip,というか女声って歌での話？それとも普通の会話とかで？,0,'というか女声って歌での話？ それとも普通の会話とかで？\t声としての話 __BR__ 歌も...


### nofindsを検証 --  => ok

In [120]:
train_nofinds

0    49
1    89
dtype: int64

In [121]:
tdf = train_df.loc[[0, 49, 89]].reset_index(drop=True)
tdf, _, _ = recover_original_text(tdf, corpus_df)

100%|██████████| 3/3 [00:24<00:00,  8.26s/it]


In [122]:
tdf

,id,source,text,label,original_text
0,80074aa43,news4vip,まともに相手されてない人との関係なんて\nそんな大事にするものか？,0,'まともにLINEの返信してくれないんだけど諦めなきゃダメだろ __BR__ フラれたらギク...
1,c32c6dbb4,news4vip,というか女声って歌での話？それとも普通の会話とかで？,0,'というか女声って歌での話？ それとも普通の会話とかで？\t声としての話 __BR__ 歌も...
2,b76fd6fcc,news4vip,興味ないけど\r\nBLOOD+の小夜かディーヴァか\r\nあと女性のコスプレはルックスに・...,0,'八紘一宇に共感するファシストどもや、 __BR__ 自衛隊合憲論という詭弁を弄して改憲運動...


In [142]:
test_nofinds

0     2
1     5
2    34
3    44
4    95
dtype: int64

In [145]:
tdf = test_df.loc[[2, 5, 34 ,44, 95]].reset_index(drop=True)
tdf, _, _ = recover_original_text(tdf, corpus_df)

100%|██████████| 5/5 [03:00<00:00, 36.01s/it]


In [155]:
tdf

,id,source,text,original_text
0,004674725,livejupiter,そうなんやろなあ色々と勿体ない感じしたわ\n終わり方と黒幕キャラは好きやったで\n\nちなワ...,'ウォッチメンてマーベルだっけか __BR__ 原作ファンの評価はいいらしいけど消化不良起こ...
1,005c3e2a1,news4vip,昔濃霧に潜む毒蛇と戦い封印した能力者たちのこと\r\n黒笹の黒笹蒼斗や霧蛇奇様をイメージする...,'能力名よりもキャラ設定についての感想が欲しい\t最初に終ノ段に至った六人の能力者の一人って...
2,0288195ba,news4vip,男遊び人\r\nアルテマウェポン\r\nマサムネブレード\r\n終焉の者の巨剣,'男遊び人 __BR__ アルテマウェポン __BR__ マサムネブレード __BR__ 終...
3,030675435,livejupiter,ええねん。こないだバネキもっとる状態で単発でキュンでたから人生そう甘くないねんな,'わい8500円を飛ぶに捨てた模様wwwww\tニキはつよい子や…\tええねん。こないだバネ...
4,07341b31f,news4vip,え、こたえどれなん？(´゜∀゜`；)\n酔っ払っててよく読めてない\n\n砂糖入ってるのは飲...,'一週間にひとり平均2リットルだと __BR__ 4人家族で1ヶ月32リットルのウーロン茶を...
